In [39]:
import pandas as pd

# Specify relative path to the file
file_path = "../IAPS/banco_incompleto.csv"

# Detect the encoding of the file
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())
    file_encoding = result['encoding']

# Load data into a pandas DataFrame with the correct encoding and delimiter specified
df = pd.read_csv(file_path, encoding=file_encoding, delimiter=',')




In [40]:
import string

# Unique proc_max values
unique_proc_max = df['proc_max'].unique()

# Create a dictionary that maps integers to letters A-Z, AA-ZZ, etc.
def create_alpha_mapping(unique_values):
    alpha = list(string.ascii_uppercase)
    double_alpha = [a+b for a in alpha for b in alpha]
    all_alpha = alpha + double_alpha
    return {value: all_alpha[i] for i, value in enumerate(unique_values)}

alpha_mapping = create_alpha_mapping(unique_proc_max)

# Now replace the numbers in the 'proc_max' column with their corresponding letters
df['proc_max'] = df['proc_max'].map(alpha_mapping)


In [41]:
# Update the values
mapping = {
    1: 'branca',
    2: 'preta',
    3: 'parda',
    4: 'amarela',
    5: 'indígena',
    99: 'sem informação'
}

df['racacor'] = df['racacor'].map(mapping)

In [42]:
mapping = {
    1: 'cirurgia oncológica',
    2: 'quimioterapia',
    3: 'radioterapia',
    4: 'diagnóstico',
    5: 'outros de oncologia',
    6: 'outras cirurgias com os CIDs de câncer "C" ou "D"'
}

df['tipo_proc_min'] = df['tipo_proc_min'].map(mapping)
df['tipo_tto_min'] = df['tipo_tto_min'].map(mapping)

In [43]:
df['tipo_tto_min']

0       cirurgia oncológica
1             quimioterapia
2             quimioterapia
3             quimioterapia
4             quimioterapia
               ...         
4784          quimioterapia
4785          quimioterapia
4786          quimioterapia
4787           radioterapia
4788          quimioterapia
Name: tipo_tto_min, Length: 4789, dtype: object

In [44]:
df['tipo_proc_min']

0       outros de oncologia
1             quimioterapia
2       outros de oncologia
3             quimioterapia
4             quimioterapia
               ...         
4784          quimioterapia
4785          quimioterapia
4786          quimioterapia
4787    outros de oncologia
4788          quimioterapia
Name: tipo_proc_min, Length: 4789, dtype: object

In [45]:
# Define a mapping dictionary
perfil_mapping = {
    '100': 'somente cirurgia',
    '020': 'somente quimioterapia',
    '003': 'somente radioterapia',
    '123': 'cirurgia, quimio e radio',
    '120': 'cirurgia e quimio',
    '103': 'cirurgia e radio',
    '023': 'quimio e radio'
}

# Replace the values in 'perfil' column
df['perfil'] = df['perfil'].map(perfil_mapping)


In [47]:
 # Convert all columns with numeric values to appropriate data type
df = df.apply(pd.to_numeric, errors='ignore')

In [49]:
daf=df.copy()

In [52]:
df=daf.copy()

In [54]:
df

,sexo,racacor,causabas,qt_apac,qt_aih,obito,estadio,idade,tipo_proc_min,cidpri_min,...,regiao_pcn,macro_pcn,estabel_saude,cidade_estabel,estado_estabel,regiao_estabel,macro_estabel,regiao_bras_estabel,finaliquimio,finaliradio
0,1,3,28,1,4,1,4,57,4,6,...,6,0,51,5,3,1,0,1,2,2
1,1,1,28,2,2,1,4,51,5,3,...,21,12,5,1,6,10,7,1,1,2
2,1,1,28,2,8,1,2,64,4,2,...,6,0,1,5,3,1,0,1,0,2
3,0,1,28,1,1,1,4,75,5,6,...,61,0,26,5,3,1,0,1,1,2
4,1,5,28,4,1,1,3,68,5,6,...,35,9,30,17,3,6,8,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4784,1,1,148,2,0,0,4,62,5,6,...,69,12,23,33,3,18,11,1,1,2
4785,1,3,28,1,2,1,4,59,5,6,...,71,9,22,19,3,9,8,1,1,2
4786,1,1,28,5,0,1,3,67,5,2,...,55,10,5,1,6,10,7,1,0,2
4787,1,1,28,4,3,1,1,74,4,6,...,53,4,26,5,3,1,0,1,2,0


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Handle categorical variables
categorical_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col].astype(str))
# Handle missing values
imp = SimpleImputer(strategy='most_frequent')
df_imputed = imp.fit_transform(df)

# Create a new DataFrame from the imputed data
df = pd.DataFrame(df_imputed, columns=df.columns)

# Fill remaining missing values
df = df.fillna(value=0)  # or some other value that makes sense in your context


# Split data into X (features) and y (target)
X = df.drop('obito', axis=1)
y = df['obito']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Print accuracy and classification report
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


ValueError: Shape of passed values is (4789, 40), indices imply (4789, 41)